In [1]:
class get_tweets:
    tweepy = __import__('tweepy')
    pd = __import__('pandas')
    datetime = __import__('datetime')
    
    def __init__(self):
        print('get_tweets V.0.1 \nImported tweepy,pandas,datetime packages')
        
    def connect(self,consumer_key,consumer_secret,access_token,access_token_secret):
        consumer_key = consumer_key
        consumer_secret = consumer_secret
        access_token = access_token
        access_token_secret = access_token_secret
        auth = self.tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self.api = self.tweepy.API(auth)
        
    def tweet_to_df(self,tweet):
        fields = ['created_at','id', 'full_text','display_text_range', "entities",'in_reply_to_status_id',
                  'in_reply_to_user_id','coordinates','user','retweet_count','favorite_count']
        fields_dict={}
        for field in fields:
            fields_dict[field] = getattr(tweet, field)
        temp_tweet = self.pd.DataFrame.from_dict(fields_dict, orient='index').T
        return temp_tweet
    
    def get_original_tweets(self,user_name):
        user_object = self.api.get_user(screen_name=user_name)
        user_id = user_object.id_str
        tweets_df = self.pd.DataFrame()
        for tweet in self.tweepy.Cursor(self.api.user_timeline, id=user_id, tweet_mode='extended').items(200):
            if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                single_tweet=self.tweet_to_df(tweet)
                tweets_df = self.pd.concat([tweets_df, single_tweet], ignore_index=True)
        earliest_tweet = min(tweets_df['id'])
        while True:
            for tweet in self.tweepy.Cursor(self.api.user_timeline, id=user_id, max_id=earliest_tweet, tweet_mode='extended').items(200):
                if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                    single_tweet=self.tweet_to_df(tweet)
                    tweets_df = self.pd.concat([tweets_df, single_tweet], ignore_index=True)
            new_earliest = min(tweets_df['id'])
            if new_earliest == earliest_tweet:
                break
            else:
                earliest_tweet = new_earliest
            print(f'Number of original Tweets got so far:{len(tweets_df)}')
        tweets_df['hashtags'] = [d.get('hasthags') for d in tweets_df.entities]
        tweets_df['user_mentions'] = [d.get('user_mentions') for d in tweets_df.entities]
        tweets_df['followers_count'] = [d.followers_count for d in tweets_df.user]
        tweets_df.drop(['user','entities'], axis=1, inplace=True)
        return tweets_df
    
    def get_original_tweets_by_dates(self,user_name,start_date,end_date):
        user_object = self.api.get_user(screen_name=user_name)
        user_id = user_object.id_str
        tweets_df = self.pd.DataFrame()
        for tweet in self.tweepy.Cursor(self.api.user_timeline, id=user_id, tweet_mode='extended').items(200):
            if (not tweet.retweeted) and ('RT @' not in tweet.full_text) and (self.datetime.datetime(end_date['year'], end_date['month'], end_date['day']) >= tweet.created_at):
                if (self.datetime.datetime(start_date['year'], start_date['month'], start_date['day']) >= tweet.created_at):
                    break
                else:
                    single_tweet=self.tweet_to_df(tweet)
                    tweets_df = self.pd.concat([tweets_df, single_tweet], ignore_index=True)
        earliest_tweet = min(tweets_df['id'])
        while True:
            for tweet in self.tweepy.Cursor(self.api.user_timeline, id=user_id, max_id=earliest_tweet, tweet_mode='extended').items(200):
                if (not tweet.retweeted) and ('RT @' not in tweet.full_text) and (self.datetime.datetime(end_date['year'], end_date['month'], end_date['day']) >= tweet.created_at):
                    if (self.datetime.datetime(start_date['year'], start_date['month'], start_date['day']) >= tweet.created_at):
                        break
                    else:
                        single_tweet=self.tweet_to_df(tweet)
                        tweets_df = self.pd.concat([tweets_df, single_tweet], ignore_index=True)
            new_earliest = min(tweets_df['id'])
            if new_earliest == earliest_tweet:
                break
            else:
                earliest_tweet = new_earliest
            print(f'Number of original Tweets got so far:{len(tweets_df)}')
        tweets_df['hashtags'] = [d.get('hasthags') for d in tweets_df.entities]
        tweets_df['user_mentions'] = [d.get('user_mentions') for d in tweets_df.entities]
        tweets_df['followers_count'] = [d.followers_count for d in tweets_df.user]
        tweets_df.drop(['user','entities'], axis=1, inplace=True)
        return tweets_df


In [2]:
ins = get_tweets()

get_tweets V.0.1 
Imported tweepy,pandas,datetime packages


In [3]:
ins.connect('9fNwXla6gTMXBlL7P5LCbyKhU',
            '03rznfCOH4twqCkc7tZHeIXikZ4D1sY6rH3j3Fw3nESYDxHmrb',
            '977303768558718978-rXF69Ll2f6pLwHBDMg25K3yO4H8G79T',
            'UbbdQWpbBzLjOb0Z4Ef7trJG7SVXwDmYxYTRvbjSm780q')

In [4]:
pence=ins.get_original_tweets('@VP')

Number of original Tweets got so far:121
Number of original Tweets got so far:226
Number of original Tweets got so far:341
Number of original Tweets got so far:453
Number of original Tweets got so far:535
Number of original Tweets got so far:653
Number of original Tweets got so far:769
Number of original Tweets got so far:871
Number of original Tweets got so far:1012
Number of original Tweets got so far:1155
Number of original Tweets got so far:1259
Number of original Tweets got so far:1377
Number of original Tweets got so far:1488
Number of original Tweets got so far:1601
Number of original Tweets got so far:1694
Number of original Tweets got so far:1733


In [342]:
trump=ins.get_original_tweets_by_dates('@realDonaldTrump',{'year':2019,'month':1,'day':1},{'year':2019,'month':12,'day':14})

Number of original Tweets got so far:91
Number of original Tweets got so far:170
Number of original Tweets got so far:274
Number of original Tweets got so far:379
Number of original Tweets got so far:466
Number of original Tweets got so far:570
Number of original Tweets got so far:668
Number of original Tweets got so far:768
Number of original Tweets got so far:897
Number of original Tweets got so far:1033
Number of original Tweets got so far:1159
Number of original Tweets got so far:1277
Number of original Tweets got so far:1408
Number of original Tweets got so far:1535
Number of original Tweets got so far:1670
Number of original Tweets got so far:1690
Number of original Tweets got so far:1699
